<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-20 15:03:01
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 17


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  35.47 K
Current:  1.31 C
-------------------
Today PnL: 83.30 K (0.64%)
Current PnL: -18.01 L (-12.55%)
CY Booked + Current PnL: -11.64 L (-8.11%)
-------------------
Total profit:  2.94 L
Total loss:  -20.95 L
-------------------
Total Booked + Current PnL: 15.34 L (13.16%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.86%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 74.47 L (56.77%)
Deployed:  1.17 C
Current:  1.31 C
CAGR/XIRR %: 8.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LICI,827.06,983.00,-5.71,H-LC,89.25,96331.0,11144.0,4923.0,0.79,13.08,5.11,18.85,35.0,2.26,0.74,28.26,XY25,NTT,INSURANCE
62,RELIANCE,1291.83,1526.00,7.63,X-LC,58.06,159947.0,19138.0,6382.0,2.38,13.59,3.99,18.13,17.0,3.00,1.22,26.29,XY25,NTT,REFINERIES
84,UJJIVANSFB,52.77,53.00,50.53,M-SC,87.10,130464.0,-12015.0,12642.0,0.71,-8.43,9.69,0.44,242.0,-0.95,1.00,53.98,OX40N,NTT,BANKS
33,HCLTECH,1647.60,1965.74,20.93,H-LC,77.42,105957.0,5453.0,13955.0,1.35,5.43,13.17,19.31,31.0,0.39,0.81,27.87,X40,ATH,IT
78,TATAELXSI,7332.28,7332.00,-15.81,X-MC,61.29,88438.0,-14214.0,14212.0,1.56,-13.85,16.07,-0.00,57.0,-1.00,0.68,34.44,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,51.61,151956.0,1043.0,59415.0,-0.97,0.69,39.10,40.06,100.0,0.02,1.16,24.29,AR,ATH,AUTO
90,VOLTAS,1278.28,1918.49,-29.76,H-MC,66.67,191550.0,-192.0,96216.0,1.83,-0.10,50.23,50.08,99.0,-0.00,1.46,6.69,XY25,ATH,AC


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4567.52,-4.86,X-SC,97.85,88606.0,-12357.0,107798.0,4.89,-12.24,121.66,94.53,60.0,-0.11,0.68,31.37,SR,ATH,CHEMICALS
51,LAOPALA,369.40,464.00,124.01,H-SC,78.49,71269.0,-29577.0,55405.0,3.91,-29.33,77.74,25.61,131.0,-0.53,0.54,28.54,AR,NTT,CERAMICS
18,CAMS,3643.00,5250.99,3.56,H-SC,83.87,116438.0,14434.0,30588.0,3.58,14.15,26.27,44.14,125.0,0.47,0.89,32.69,X40N,ATH,MISC
47,JIOFIN,310.58,387.00,-6.92,M-LC,86.02,205625.0,-11781.0,65265.0,3.40,-5.42,31.74,24.61,94.0,-0.18,1.57,46.20,XY24,BTT,FINANCE
81,TITAGARH,1097.23,1548.00,-13.74,H-SC,25.81,151541.0,-40474.0,119354.0,3.14,-21.08,78.76,41.08,151.0,-0.34,1.16,28.26,XY24,NTT,ENGINEERING


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-353.28,H-SC,69.89,107679.0,-25607.0,110199.0,-5.00,-19.21,102.34,63.47,142.0,-0.23,0.82,80.50,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.0,7383.33,M-SC,100.00,82104.0,-11706.0,88327.0,-3.33,-12.48,107.58,81.67,238.0,-0.13,0.63,59.00,XR,NTT,CERAMICS
66,SAMMAANCAP,170.35,326.0,-159.50,M-SC,29.03,71706.0,-30504.0,123894.0,-3.19,-29.84,172.78,91.37,200.0,-0.25,0.55,16.40,XY25,NTT,FINANCE
19,CERA,8421.60,8422.0,-16.23,X-SC,73.12,86456.0,-23025.0,23032.0,-3.10,-21.03,26.64,0.00,65.0,-1.00,0.66,30.85,OX40N,NTT,CERAMICS
60,RAJESHEXPO,517.65,518.0,1670.20,M-SC,10.75,48315.0,-88862.0,88958.0,-2.64,-64.78,184.12,0.07,232.0,-1.00,0.37,20.16,OX40N,NTT,JEWELLERY


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,50.53,M-SC,87.10,130464.0,-12015.0,12642.0,0.71,-8.43,9.69,0.44,242.0,-0.95,1.00,53.98,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.0,-15.81,X-MC,61.29,88438.0,-14214.0,14212.0,1.56,-13.85,16.07,-0.00,57.0,-1.00,0.68,34.44,OX40N,NTT,IT
89,VIPIND,488.80,489.0,-934.27,H-SC,94.62,79472.0,-15844.0,15886.0,1.93,-16.62,19.99,0.04,153.0,-1.00,0.61,56.75,OX40N,NTT,MISC
76,SYMPHONY,1306.42,1306.0,-33.55,M-SC,2.15,141532.0,-29609.0,29552.0,-0.62,-17.30,20.88,-0.03,190.0,-1.00,1.08,1.72,OX40N,NTT,DURABLES
83,TTKPRESTIG,769.29,770.0,-10.66,X-SC,6.45,81751.0,-19026.0,19122.0,0.61,-18.88,23.39,0.09,64.0,-0.99,0.62,6.38,OX40N,NTT,DURABLES


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,INFY,1461.46,2275.00,-9.55,X-LC,74.19,289998.0,28397.0,117217.0,0.11,10.85,40.42,55.67,1.0,0.24,2.22,17.62,X40,BTT,IT
80,TCS,3794.03,4998.00,-17.76,X-LC,37.63,267844.0,-28090.0,122003.0,0.29,-9.49,45.55,31.73,2.0,-0.23,2.05,7.21,X40,BTT,IT
36,HINDUNILVR,2413.81,2723.00,-18.08,X-LC,32.26,257174.0,-13173.0,47809.0,0.09,-4.87,18.59,12.81,5.0,-0.28,1.97,6.44,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,11.96,X-LC,55.91,256772.0,27539.0,39748.0,0.43,12.01,15.48,29.35,7.0,0.69,1.96,22.01,XY25,ATH,FMCG
57,NESTLEIND,2268.60,2755.00,-14.62,X-LC,41.94,271311.0,5885.0,51034.0,-0.94,2.22,18.81,21.44,10.0,0.12,2.07,8.55,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-5.13,M-LC,26.88,95339.0,16977.0,17056.0,0.36,21.66,17.89,43.43,77.0,1.0,0.73,28.14,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,3.56,H-SC,83.87,116438.0,14434.0,30588.0,3.58,14.15,26.27,44.14,125.0,0.47,0.89,32.69,X40N,ATH,MISC
34,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,51.61,151956.0,1043.0,59415.0,-0.97,0.69,39.10,40.06,100.0,0.02,1.16,24.29,AR,ATH,AUTO
92,WIPRO,243.46,420.00,-3.36,M-LC,81.72,165447.0,14502.0,94950.0,0.48,9.61,57.39,72.51,68.0,0.15,1.26,13.97,XR,NTT,IT
3,ADANIPORTS,1103.69,1583.00,-5.13,M-LC,26.88,95339.0,16977.0,17056.0,0.36,21.66,17.89,43.43,77.0,1.00,0.73,28.14,XY24,NTT,MISC
85,UNIONBANK,123.87,163.00,-6.44,M-LC,63.44,162534.0,21694.0,22804.0,2.34,15.40,14.03,31.59,86.0,0.95,1.24,41.19,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-31.76,M-SC,70.97,198979.0,27358.0,134709.0,1.83,15.94,67.70,94.43,194.0,0.20,1.52,46.86,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-3.36,M-LC,81.72,165447.0,14502.0,94950.0,0.48,9.61,57.39,72.51,68.0,0.15,1.26,13.97,XR,NTT,IT
41,INDIAMART,2327.09,4850.92,-49.41,H-SC,79.57,131673.0,8337.0,125432.0,1.74,6.76,95.26,108.45,122.0,0.07,1.01,30.62,AR,ATH,MISC
34,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,51.61,151956.0,1043.0,59415.0,-0.97,0.69,39.10,40.06,100.0,0.02,1.16,24.29,AR,ATH,AUTO
12,BANKINDIA,116.91,190.00,-34.31,M-MC,46.24,173886.0,-5922.0,118329.0,1.28,-3.29,68.05,62.52,170.0,-0.05,1.33,29.67,XR,NTT,BANKS


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-57.75,L-SC,62.37,63706.0,-49843.0,89896.0,1.50,-43.90,141.11,35.27,260.0,-0.55,0.49,55.92,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.00,7383.33,M-SC,100.00,82104.0,-11706.0,88327.0,-3.33,-12.48,107.58,81.67,238.0,-0.13,0.63,59.00,XR,NTT,CERAMICS
54,MASFIN,326.60,402.39,-23.67,H-SC,80.65,87000.0,-10980.0,33721.0,0.57,-11.21,38.76,23.21,141.0,-0.33,0.66,26.18,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4567.52,-4.86,X-SC,97.85,88606.0,-12357.0,107798.0,4.89,-12.24,121.66,94.53,60.0,-0.11,0.68,31.37,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,-5.13,M-LC,26.88,95339.0,16977.0,17056.0,0.36,21.66,17.89,43.43,77.0,1.00,0.73,28.14,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7383.33,M-SC,100.00,82104.0,-11706.0,88327.0,-3.33,-12.48,107.58,81.67,238.0,-0.13,0.63,59.00,XR,NTT,CERAMICS
48,JPPOWER,18.73,26.20,-35.73,L-SC,98.92,131089.0,-10753.0,67327.0,1.76,-7.58,51.36,39.88,258.0,-0.16,1.00,32.85,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4567.52,-4.86,X-SC,97.85,88606.0,-12357.0,107798.0,4.89,-12.24,121.66,94.53,60.0,-0.11,0.68,31.37,SR,ATH,CHEMICALS
77,TANLA,917.30,1963.11,-30.98,H-SC,96.77,169272.0,-70143.0,343097.0,0.82,-29.30,202.69,114.01,128.0,-0.20,1.29,55.25,AR,ATH,IT
69,SFL,1016.35,1287.00,323.87,H-SC,95.70,191462.0,-70756.0,140591.0,0.09,-26.98,73.43,26.63,145.0,-0.50,1.46,19.04,XY24,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.31
1,25,42.70
2,50,71.44


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.43
LC    38.20
MC    21.64
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.12
XY25     14.11
X40      12.59
X40N     10.80
XR       10.26
OX40N     8.30
AR        7.98
X5K       2.20
X200      1.47
SR        1.18
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.35
X-LC    15.54
M-SC    13.98
H-LC    13.27
H-MC     8.48
M-LC     8.29
X-MC     7.58
M-MC     5.58
X-SC     2.73
L-SC     2.37
L-LC     1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.47,-3.35,37.93
IT,12.20,-11.31,66.58
BANKS,7.99,-9.21,53.78
FINANCE,7.68,-19.91,63.31
MISC,6.67,-24.12,72.53
PAINTS,5.57,-19.81,43.42
ELECTRICAL,5.08,-5.68,51.44
HEALTHCARE,4.68,-5.46,36.27
AUTO,4.50,-14.98,65.54


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2862671.0,25
XR,1116491.0,13
AR,1025207.0,9
X40,672638.0,9
XY25,540138.0,10
X40N,516459.0,10
OX40N,344824.0,11
SR,191463.0,2
X5K,109955.0,3


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1964427.0,19
M-SC,1500757.0,18
X-LC,630860.0,9
H-LC,618162.0,12
M-MC,614548.0,5
H-MC,512389.0,7
X-MC,426241.0,6
M-LC,359021.0,6
L-SC,287465.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,908558.0,6
M-SC,XY24,604853.0,5
H-SC,AR,523934.0,3
M-MC,XY24,405442.0,3
M-SC,XR,342941.0,4
X-LC,X40,323276.0,3
M-LC,XY24,246388.0,4
H-LC,X40N,245481.0,5
H-MC,XY24,245065.0,2


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
